In [2]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using PyCall
using JLD2
using LinearAlgebra
using NearestNeighbors
using Rotations
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMol
  ✓ MorphoMol
  1 dependency successfully precompiled in 3 seconds. 243 already precompiled.
[ Info: Precompiling MorphoMol [85e20460-a9b2-48f6-9df6-e154e9748d83]


In [3]:
function get_persistence_diagram(points)
    py"""
    import oineus as oin
    import numpy as np
    import torch
    import diode

    def get_persistence_diagram(points):
        points = np.asarray(points)
        simplices = diode.fill_alpha_shapes(points)
        fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])

        dcmp = oin.Decomposition(fil, True)
        params = oin.ReductionParams()
        dcmp.reduce(params)
        dgm = dcmp.diagram(fil, include_inf_points=False)
        return dgm
    """
    py"get_persistence_diagram"(points)
end

get_persistence_diagram (generic function with 1 method)

In [4]:
function get_tps(dgms)
    [MorphoMol.Energies.get_total_persistence(dgm) for dgm in dgms]
end
    
function get_fsols(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[6]) for i in 1:n]
end

function get_thetas(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[7]) for i in 1:n]
end

function txt_to_points(input_file)
    [[parse(Float64, s) for s in split(p," ")] for p in readlines(input_file)[3:end]]
end

txt_to_points (generic function with 1 method)

In [5]:
function get_configurations_from_folder(folder)
    files = readdir(folder)
    realizations = [Vector{Vector{Vector{Float64}}}([]) for i in 1:length(files)]
    for file in readdir(folder)
        centers = txt_to_points("$(folder)$(file)")
        n_atoms_per_mol = Int(length(centers) / 2)
        c1 = centers[1:n_atoms_per_mol]
        c2 = centers[n_atoms_per_mol+1:end]
        realizations[parse(Int, split(file, ".")[1]) + 1] = [c1, c2]
    end
    realizations
end

get_configurations_from_folder (generic function with 1 method)

In [6]:
function folder_of_txts_to_jld2_with_persistence_computation(folder)
    mol_type = split(folder, "/")[end-2]
    idx = split(folder, "/")[end-1]
    py_dgms = MorphoMol.Energies.get_interface_diagrams(folder)
    dgms = [[dgm[1], dgm[2], dgm[3], dgm[4]] for dgm in py_dgms]
    tps = get_tps(dgms)
    Es = get_fsols(folder)
    thetas = get_thetas(folder)
    realizations = get_configurations_from_folder(folder)
    @save "../../Data/measure_paths/jld2s/$(mol_type)/$(idx).jld2" dgms tps Es thetas realizations
end

folder_of_txts_to_jld2_with_persistence_computation (generic function with 1 method)

In [75]:
# folder = "../../Data/measure_paths/txts/6r7m/"
# for subdir in readdir(folder)
#     if subdir != ".DS_Store"
#         folder_of_txts_to_jld2_with_persistence_computation("$(folder)$(subdir)/")
#     end
# end

# folder = "../../Data/measure_paths/txts/6r7m/1/"
# dgms = get_interface_diagrams(folder)
# xs = [i for i in 1:length(dgms)]
# dgms_selection = [[dgm[2]; dgm[3]] for dgm in dgms]
# tps = get_tps(dgms_selection)
# Es = get_fsols(folder)
# thetas = get_thetas(folder);

In [51]:
# @load "../../Data/collected_simulation_results/rwm_ma_2_6r7m/52.jld2" input output
# realizations = [[e for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization(state, input["template_centers"])...))] for state in output["states"]]
# pdgms = [get_persistence_diagram(r) for r in realizations]
# dgms = [[dgm[1], dgm[2], dgm[3]] for dgm in pdgms]
# merge!(output, Dict("PDGMs" => dgms))
# merge!(input, Dict("mol_type" => "6r7m"))
# @save "../../Data/examples/correct_assembly_ma_only.jld2" input output

In [9]:
@load "../../Data/examples/correct_assembly_ma_only.jld2" input output

realizations = [[e for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization(state, input["template_centers"])...))] for state in output["states"]]
dgms = output["PDGMs"]
Es = output["Es"]
persistence_weight = -0.1
realizations = [MorphoMol.Utilities.get_matrix_realization(state, input["template_centers"]) for state in output["states"]]
cdgms2 = [dgm[2] for dgm in dgms]
cdgms3 = [dgm[3] for dgm in dgms]
cp2s = [MorphoMol.Energies.get_persistence(dgm) for dgm in cdgms2]
cp3s = [MorphoMol.Energies.get_persistence(dgm) for dgm in cdgms3]
mol_type = input["mol_type"]    
exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
thetas = [MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, state) for state in output["states"]]
xs = [i for i in 1:length(dgms)];
zero_ps = [MorphoMol.Energies.get_persistence(dgm[1], 0.1) for dgm in dgms]
one_ps = [MorphoMol.Energies.get_persistence(dgm[2], -0.1) for dgm in dgms]
two_ps = [MorphoMol.Energies.get_persistence(dgm[3], -0.1) for dgm in dgms]
tps = [MorphoMol.Energies.get_total_persistence(dgm, [0.1, -0.1, -0.1, 0.0]) for dgm in dgms];

In [12]:
f = Figure(fontsize = 7)

#Slider
sl_i = Slider(f[4, 1:4], range = 1:length(dgms), startvalue = 1)
x = sl_i.value

# Persistence Diagrams
pd_ax_a = Axis(f[1, 1], title = "Persistence codim = 2", xticks = 0:25:100, yticks = 0:25:100)
pd_ax_b = Axis(f[1, 2], title = "Persistence codim = 3", xticks = 0:25:100, yticks = 0:25:100)
dgm_points_2 = @lift([Point2f(dgms[$x][2][i,1], dgms[$x][2][i,2]) for i in 1:size(dgms[$x][2])[1]])
dgm_points_3 = @lift([Point2f(dgms[$x][3][i,1], dgms[$x][3][i,2]) for i in 1:size(dgms[$x][3])[1]])

ms = 5
scatter!(pd_ax_a, dgm_points_2, color = :orange, markersize = ms)
lines!(pd_ax_a, [0, 100], [0, 100], color=:black)
scatter!(pd_ax_b, dgm_points_3, color = :green, markersize = ms)
lines!(pd_ax_b, [0, 100], [0, 100], color=:black)

#Energy and Theta
theta_ax = Axis(f[2, 1], title = L"\Theta")
E_ax = Axis(f[2, 2], title = L"F_{sol}")

theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
E_mark = @lift(Point2f($x, $@lift(Es[$x])))

scatter!(theta_ax, xs, thetas, color=:magenta, markersize = ms)
scatter!(theta_ax, theta_mark, color=:black, markersize = 10)

scatter!(E_ax, xs, Es, color=:blue, markersize = ms)
scatter!(E_ax, E_mark, color=:black, markersize = 10)

# Configuration
conf_ax = Axis3(f[1:2, 3:4])
c1_points = @lift([Point3f(realizations[$x][1][1,i], realizations[$x][1][2,i], realizations[$x][1][3,i]) for i in 1:size(realizations[$x][1])[2]])
c2_points = @lift([Point3f(realizations[$x][2][1,i], realizations[$x][2][2,i], realizations[$x][2][3,i]) for i in 1:size(realizations[$x][2])[2]])

conf_ms = 10
scatter!(conf_ax, c1_points, markersize = conf_ms)
scatter!(conf_ax, c2_points, markersize = conf_ms)

#Persistence Measures
zero_p_ax = Axis(f[3, 1], title = "0.1 * total 0 persistence")
one_p_ax = Axis(f[3, 2], title = "-0.1 * total 1 persistence")
two_p_ax = Axis(f[3, 3], title = "-0.1 * total 2 persistence")
tp_ax = Axis(f[3, 4], title = "Total Persistences Summed")

tp_mark = @lift(Point2f($x, $@lift(tps[$x])))

zero_p_mark = @lift(Point2f($x, $@lift(zero_ps[$x])))
one_p_mark = @lift(Point2f($x, $@lift(one_ps[$x])))
two_p_mark = @lift(Point2f($x, $@lift(two_ps[$x])))

scatter!(zero_p_ax, xs, zero_ps, color=:orange, markersize = ms)
scatter!(zero_p_ax, zero_p_mark, color=:black, markersize = 10)

scatter!(one_p_ax, xs, one_ps, color=:orange, markersize = ms)
scatter!(one_p_ax, one_p_mark, color=:black, markersize = 10)

scatter!(two_p_ax, xs, two_ps, color=:green, markersize = ms)
scatter!(two_p_ax, two_p_mark, color=:black, markersize = 10)

scatter!(tp_ax, xs, tps, color=:red, markersize = ms)
scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

display(f)

GLMakie.Screen(...)